<a href="https://colab.research.google.com/github/tomersk/python-applications/blob/main/01_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Reading Netcdf file for each year

In [15]:
from google.colab import drive
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np

drive.mount("/content/gdrive/")

file = '/content/gdrive/My Drive/Colab Notebooks/IMD/rain_1901.nc'




Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [16]:
ds = nc.Dataset(file)
print(ds)

# print the dimensions of the nc file
print(ds.dimensions)
print(ds.variables)
print("======= Variable names on seperate line =====")
# print one by one
for var in ds.variables:
  print(var)


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    dimensions(sizes): lat(129), lon(135), time(365)
    variables(dimensions): int32 time(time), float64 lon(lon), float64 rain(lat, lon, time), float64 lat(lat)
    groups: 
{'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 129, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 135, 'time': <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 365}
{'time': <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: days since 1901-01-01 00:00:00
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (365,)
filling on, default _FillValue of -2147483647 used, 'lon': <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    _FillValue: nan
unlimited dimensions: 
current shape = (135,)
filling on, 'rain': <class 'netCDF4._netCDF4.Variable'>
float64 rain(lat, lon, time)
    long_name: rainfall
    units: mm/day
    _FillValue: n

In [17]:
latitude=ds.variables['lat'][:]
longitude=ds.variables['lon'][:]
day=ds.variables['time'][:]
rainfall=ds.variables['rain'][:]

In [18]:
ds.close()

2. Visualizing Data- 1 Day rainfall plot

In [ ]:
#plt.bar(day,rainfall[60,60,:])
plt.imshow(rainfall[:,:,1])
plt.colorbar()
plt.xticks(longitude)
plt.gca().invert_yaxis()
print(max(rainfall[80,65,:]))


In [ ]:
len(longitude)